### Chargement du fichier joblib conbtenant la regression lineaire

In [1]:
import joblib

model_path = 'regression.joblib'
model = joblib.load(model_path)
model

/home/antoine/.local/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator LinearRegression from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


LinearRegression()

### Recuperer les valeurs des coefficients

In [2]:
coeffs, origin = list(model.coef_), model.intercept_
print(coeffs, origin)

[1000.0000000000005, 10000.000000000005, 19999.999999999993] -5.820766091346741e-11


In [40]:
model.predict([[-9, -5, 67]])

array([1281000.])

### Generer une chaine de caractere contenant le code C permettant de calculer la prediction du modele

In [59]:
def produce_linear_prediction_c_code(model):
    prediction_code = "float thetas[] = {"

    # On ajoute un par un les coefficients à notre string de déclaration d'array C 'thetas'.
    for i in range(len(model.coef_)):
        prediction_code += str(model.coef_[i])
        if i < len(model.coef_) - 1:
            prediction_code += ','
    prediction_code +=  ', ' + str(model.intercept_) + '};'

  # On créé la string correspondant au code de la fonction ecrite en C,
  # en injectant le code de la regression lineaire.
    s = f"""\
float prediction(float* features, int n_features)
    {{
        {prediction_code}
        long int waited = sizeof(thetas) / sizeof(float);
        if (waited != n_features + 1)
        {{
            printf("Number of features given not coherent with the model\\n The model wait for %li values, but %i were given\\n", waited - 1, n_features);
            exit(1);
        }}
        float to_return = 0;
        for (size_t i = 0; i < n_features; i++)
        {{
            to_return += features[i] * thetas[i];
        }}

        to_return += thetas[{str(len(model.coef_))}];
        return to_return;
    }}
    """

    return s


In [60]:
print(produce_linear_prediction_c_code(model))

float prediction(float* features, int n_features)
    {
        float thetas[] = {1000.0000000000005,10000.000000000005,19999.999999999993, -5.820766091346741e-11};
        long int waited = sizeof(thetas) / sizeof(float);
        if (waited != n_features + 1)
        {
            printf("Number of features given not coherent with the model\n The model wait for %li values, but %i were given\n", waited - 1, n_features);
            exit(1);
        }
        float to_return = 0;
        for (size_t i = 0; i < n_features; i++)
        {
            to_return += features[i] * thetas[i];
        }

        to_return += thetas[3];
        return to_return;
    }
    


In [61]:
def generate_linear_c_main():
    '''
    Le main doit doit etre appele avec les valeurs constituant l'array sur laquelle on souhaite predire:
    Si l'executable s'appelle prediction:
        
            ./prediction 1 2 3
            
    Sera equivalant au code python model.predict([[1, 2, 3]])
    '''

    s = f"""\
int main(int argc, char **argv)
    {{   
        if (argc == 1)
        {{
            printf("Missing data to make a prediction\\n");
            return 1;
        }}
        float data[argc];
        for (size_t i = 1; i < argc; i++)
        {{
            data[i - 1] = atoi(argv[i]);
        }}
        printf("%f\\n", prediction(data, argc - 1));
        return 0;
    }}
    """
    return s

In [62]:
print(generate_linear_c_main())

int main(int argc, char **argv)
    {   
        if (argc == 1)
        {
            printf("Missing data to make a prediction\n");
            return 1;
        }
        float data[argc];
        for (size_t i = 1; i < argc; i++)
        {
            data[i - 1] = atoi(argv[i]);
        }
        printf("%f\n", prediction(data, argc - 1));
        return 0;
    }
    


### Fonction qui genere un fichier c contenant la fonction pour calculer la prediction et le main dans lequel on appelle cette fonction.

In [63]:
def generate_file(model):
    f = open("code.c", "w+")
    f.write(f"""\
    #include <stddef.h>
    #include <stdio.h>
    #include <stdlib.h>
    #include <string.h>
    
    {produce_linear_prediction_c_code(model)}
    {generate_linear_c_main()}
    """)
    f.close()
    print("gcc code.c -o prediction")


In [64]:
generate_file(model)

gcc code.c -o prediction


In [38]:
from sklearn.linear_model import LinearRegression
import numpy as np

In [65]:
'''
Exemple took in the sklearn documentation
'''
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
# y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2])) + 3
reg = LinearRegression().fit(X, y)

In [67]:
generate_file(reg)

gcc code.c -o prediction


In [68]:
reg.predict([[3, 5]])

array([16.])

En executant le code C généré, on obtient aussi 16.